In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # for visualiation
from datetime import datetime
pd.options.display.max_columns = None

In [4]:

# combined mass shooting data from 13-18
mass = pd.read_csv('./data/mass_shooting_data_13_18.csv')
mass = mass.drop(['Operations'], axis=1)

In [5]:
df = pd.read_csv('./data/gunsales_population.csv')
df=df.drop('Unnamed: 0',axis=1)

In [6]:
mass.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured
0,"December 31, 2013",New York,Brooklyn,60 Glenmore Ave,0,6
1,"December 28, 2013",Alabama,Montgomery,954 Highland Ave,3,5
2,"December 26, 2013",Louisiana,Slidell,2144 First St,2,6
3,"December 26, 2013",Louisiana,Lockport,313 Tenth St,3,3
4,"December 25, 2013",New Jersey,Irvington,Nye Avenue and 21st Street,3,2


In [7]:
df.head()

,year,state,population,admin,handgun,long_gun,month,multiple,other,permit,totals
0,2013,United States,316234505.0,267.0,881038.0,788328.0,1,31029.0,26137.0,693076.0,2483230.0
1,2013,United States,316234505.0,375.0,770872.0,703223.0,2,31163.0,22709.0,664252.0,2298561.0
2,2013,United States,316234505.0,376.0,683266.0,700873.0,3,23878.0,23817.0,695386.0,2197116.0
3,2013,United States,316234505.0,665.0,533674.0,554003.0,4,18624.0,21136.0,517224.0,1702455.0
4,2013,United States,316234505.0,619.0,445522.0,436674.0,5,16057.0,15605.0,449993.0,1424450.0


In [8]:
mass.columns = ['Date','state','city','Address','killed','injured']
mass = mass.drop('Address',axis=1)
mass['year']=pd.DatetimeIndex(mass['Date']).year
mass['month']=pd.DatetimeIndex(mass['Date']).month
mass = mass.drop('Date',axis=1)

In [9]:
by_shootings = df.merge(mass)

In [10]:
by_shootings.to_csv('./data/gunsales_population_by_shootings.csv',index=False)

In [11]:
def keep(x):
    if x >10:
        return 1
    else:
        return 0

In [12]:
mass_grouped = mass
mass_grouped['affected'] = mass_grouped.killed+mass_grouped.injured
mass_grouped['affected_over_10'] = mass_grouped.affected.apply(lambda x:keep(x))
mass_grouped = mass_grouped.groupby(['state','year','month'])['killed','injured','affected','affected_over_10'].agg(['sum','count','max']).reset_index()
mass_grouped.columns = ['state','year','month','killed','kill_count','killed_max','injured','incidents','injured_max','affected_sum','affected_count','affected_max','affected_over_10','affected_over_10_count','affected_over_10_max']
mass_grouped = mass_grouped.drop(['kill_count','affected_count','affected_sum','affected_count','affected_over_10_count','affected_over_10_max'],axis=1)

In [13]:
replace = mass_grouped
mass_grouped = mass_grouped.reset_index()
for yr in range(2013,2018):
    for m in range(1,13):
        kill = 0
        km = 0
        inj = 0
        inc=0
        im = 0
        aff = 0
        aff_10 = 0
        for st in mass_grouped.state.unique():
            temp = mass_grouped[(mass_grouped.state == st) & (mass_grouped.year==yr) & (mass_grouped.month ==m)]
            for ind in temp['index'].unique():
                cur = temp[temp['index']==ind]
                kill += cur.killed.values[0]
                inj += cur.injured.values[0]
                aff_10+=cur.affected_over_10.values[0]
                inc += cur.incidents.values[0]
                if cur.killed_max.values[0] >km:
                    km = cur.killed_max.values[0]
                if cur.injured_max.values[0] >im:
                    im = cur.injured_max.values[0]
                if cur.affected_max.values[0] >aff:
                    aff = cur.affected_max.values[0]
        replace = replace.append({'state':'United States',
                             'killed':kill,
                             'killed_max':km,
                             'injured':inj,
                             'incidents':inc,
                             'injured_max': im,
                             'year':yr,
                             'month':m,
                             'affected_max':aff, 
                             'affected_over_10':aff_10},ignore_index=True)
mass_grouped = replace

In [14]:
df = pd.merge(df,mass_grouped,how='left')
df = df.fillna(0)

In [15]:
df['killed_ave'] = df.killed/df.incidents
df['injured_ave'] =  df.injured/df.incidents

In [16]:
df['admin_pop'] = 100000 * df.admin/df.population
df['handgun_pop'] = 100000 * df.handgun/df.population
df['long_gun_pop'] = 100000 * df.long_gun/df.population
df['multiple_pop'] = 100000 * df.multiple/df.population
df['other_pop'] = 100000 * df.other/df.population
df['permit_pop'] = 100000 * df.permit/df.population
df['totals_pop'] = 100000 * df.totals/df.population
df['killed_pop'] = 100000 * df.killed/df.population
df['injured_pop'] = 100000 * df.injured/df.population
df['incidents_pop'] = 100000 * df.incidents/df.population

In [17]:
df

,year,state,population,admin,handgun,long_gun,month,multiple,other,permit,totals,killed,killed_max,injured,incidents,injured_max,affected_max,affected_over_10,killed_ave,injured_ave,admin_pop,handgun_pop,long_gun_pop,multiple_pop,other_pop,permit_pop,totals_pop,killed_pop,injured_pop,incidents_pop
0,2013,United States,316234505.0,267.0,881038.0,788328.0,1,31029.0,26137.0,693076.0,2483230.0,16.0,5.0,39.0,12.0,6.0,7.0,0.0,1.333333,3.250000,0.084431,278.602741,249.285890,9.812022,8.265069,219.165205,785.249541,0.005060,0.012333,0.003795
1,2013,United States,316234505.0,375.0,770872.0,703223.0,2,31163.0,22709.0,664252.0,2298561.0,14.0,4.0,45.0,12.0,8.0,8.0,0.0,1.166667,3.750000,0.118583,243.765936,222.373899,9.854396,7.181063,210.050450,726.853320,0.004427,0.014230,0.003795
2,2013,United States,316234505.0,376.0,683266.0,700873.0,3,23878.0,23817.0,695386.0,2197116.0,25.0,6.0,75.0,19.0,13.0,13.0,1.0,1.315789,3.947368,0.118899,216.063076,221.630780,7.550726,7.531436,219.895675,694.774278,0.007906,0.023717,0.006008
3,2013,United States,316234505.0,665.0,533674.0,554003.0,4,18624.0,21136.0,517224.0,1702455.0,26.0,6.0,52.0,17.0,5.0,7.0,0.0,1.529412,3.058824,0.210287,168.758940,175.187398,5.889300,6.683648,163.557105,538.352069,0.008222,0.016443,0.005376
4,2013,United States,316234505.0,619.0,445522.0,436674.0,5,16057.0,15605.0,449993.0,1424450.0,18.0,5.0,99.0,24.0,19.0,19.0,1.0,0.750000,4.125000,0.195741,140.883424,138.085501,5.077561,4.934629,142.297249,450.441042,0.005692,0.031306,0.007589
5,2013,United States,316234505.0,444.0,394363.0,392866.0,6,14526.0,13555.0,398792.0,1270817.0,25.0,6.0,131.0,31.0,9.0,9.0,0.0,0.806452,4.225806,0.140402,124.705873,124.232490,4.593427,4.286376,126.106416,401.859057,0.007906,0.041425,0.009803
6,2013,United States,316234505.0,519.0,391016.0,376846.0,7,14339.0,12468.0,416312.0,1271571.0,30.0,7.0,106.0,31.0,6.0,7.0,0.0,0.967742,3.419355,0.164119,123.647481,119.166629,4.534293,3.942644,131.646608,402.097488,0.009487,0.033519,0.009803
7,2013,United States,316234505.0,582.0,428058.0,497092.0,8,17092.0,13486.0,391909.0,1408468.0,32.0,4.0,84.0,25.0,5.0,8.0,0.0,1.280000,3.360000,0.184041,135.360940,157.190943,5.404850,4.264557,123.929867,445.387198,0.010119,0.026563,0.007906
8,2013,United States,316234505.0,469.0,413380.0,523292.0,9,16018.0,14005.0,357722.0,1392423.0,37.0,11.0,110.0,28.0,12.0,14.0,2.0,1.321429,3.928571,0.148308,130.719448,165.475934,5.065228,4.428675,113.119218,440.313431,0.011700,0.034784,0.008854
9,2013,United States,316234505.0,675.0,422005.0,578973.0,10,17954.0,14272.0,565281.0,1679099.0,18.0,5.0,80.0,18.0,12.0,13.0,1.0,1.000000,4.444444,0.213449,133.446855,183.083437,5.677432,4.513106,178.753738,530.966411,0.005692,0.025298,0.005692


In [21]:
df.isna().sum()

year                0
state               0
population          0
admin               0
handgun             0
long_gun            0
month               0
multiple            0
other               0
permit              0
totals              0
killed              0
killed_max          0
injured             0
incidents           0
injured_max         0
affected_max        0
affected_over_10    0
killed_ave          0
injured_ave         0
admin_pop           0
handgun_pop         0
long_gun_pop        0
multiple_pop        0
other_pop           0
permit_pop          0
totals_pop          0
killed_pop          0
injured_pop         0
incidents_pop       0
dtype: int64

In [20]:
df=df.fillna(0)

In [22]:
df.to_csv('./data/shootings_and_sales_per_pop.csv',index=False)